In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainDatasetPath = '/content/drive/My Drive/data/train/'
testDatasetPath = '/content/drive/My Drive/data/test/'
test_noiseDataSetPath = '/content/drive/My Drive/data/test_noise/'
mtec_testDataSetPath = '/content/drive/My Drive/data/mtec_test/'
csvPath = '/content/drive/My Drive/data/ImageName_with_Class.csv'

In [ ]:
import os
import torch
import torchvision
import pandas as pd
import torch.nn as nn
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import numpy as np
import random
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
from skimage import io
import scikit
import sklearn.metrics as metrics

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batchSize = 16
num_epochs = 5

In [ ]:
class CreateDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(csv_file, names=['Image Name', 'Image Class'])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
class DataSubset(Dataset):
  def __init__(self, subset, transform=None, target_transform=None):
        self.subset = subset
        self.transform = transform
        self.target_transform = target_transform

  def __len__(self):
        return len(self.subset)

  def __getitem__(self, index):
        image, label = self.subset[index]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
train_transforms = transforms.Compose ([transforms.ToPILImage(),
                                        transforms.Resize((220,200)),
                                        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
                                       transforms.RandomRotation(10),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()])

test_transforms =  transforms.Compose ([transforms.Resize((220,200)),
                                      transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
                                      transforms.ToTensor()])

val_transforms = test_transforms

In [ ]:
dataset = CreateDataset(csv_file = csvPath, img_dir = trainDatasetPath)
trainLength = int(len(dataset) * 0.8)
valLength = len(dataset) - trainLength

#print(len(dataset))
#print(trainLength)
#print(valLength)

trainDataset , valDataset = torch.utils.data.random_split(dataset, [trainLength, valLength])

trainingData = DataSubset(subset = trainDataset)
validationData = DataSubset(subset = valDataset, transform = val_transforms)

In [ ]:
trainDataloader = torch.utils.data.DataLoader(dataset = trainingData, batch_size = batchSize, shuffle = True)
valDataloader = torch.utils.data.DataLoader(dataset = validationData, batch_size = batchSize, shuffle = True)
get_mean_and_std(trainDataloader)

RuntimeError: ignored

In [ ]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    print(mean)
    print(std)

In [ ]:
# Model
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train Network
for epoch in range(num_epochs):
  losses = []

    for batch_idx, (data, targets) in enumerate(trainDataloader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")


TypeError: ignored

In [ ]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

In [ ]:
def ROC_curve(loader, model):
  model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            fpr, tpr, thresholds = metrics.roc_curve(y,predictions)
            plt.plot(fpr,tpr)
            plt.show()
print("F1 score on the training set")
ROC_curve(trainDataloader,model)

In [ ]:
def f1_score(loader, model,average = 'micro', digits = 2, suffix = False):
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)